# Utiliser résultats yolo

## Imports et installation

In [ ]:
!pip install sentence_transformers
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=f0bad66ca7c1929890719d3ca2325a17ec00a3279d31a6215b784df517e9cc27
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pandas as pd
import json
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
from sentence_transformers import SentenceTransformer

from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten,Reshape,Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D,UpSampling2D,InputLayer,ReLU,BatchNormalization
from tensorflow.keras.optimizers import SGD,Adam, AdamW
from tensorflow.keras.utils import to_categorical
import matplotlib
import matplotlib.pyplot as plt
import scipy
import scipy.ndimage
import numpy as np
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import pandas as pd

## Récupération des données depuis le drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
dic = {}
for i in range(20):
 path = "/content/gdrive/MyDrive/Projet_Multimedia/download/yolo_json/objets_videos_mod_" + str(i) + ".json"
 file = open(path)
 with open(path, 'r') as f:
  dictionnary = json.load(f)
 dic = dic | dictionnary
len(dic)

for i in range(2):
 path = "/content/gdrive/MyDrive/Projet_Multimedia/download/yolo_json/objets_videos_rest_0"+str(i)+".json"
 file = open(path)
 with open(path, 'r') as f:
  dictionnary = json.load(f)
 dic = dic | dictionnary
path = "/content/gdrive/MyDrive/Projet_Multimedia/download/yolo_json/objets_videos_rest_final.json"
file = open(path)
with open(path, 'r') as f:
 dictionnary = json.load(f)
dic = dic | dictionnary
len(dic)

# récup les labels
path = "/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/NLP.csv"
df_lab = pd.read_csv(path)
df = df_lab

## Embeddings avec BERT

In [ ]:
# recupérer les mots
dic_obj = {}
dic_obj_u = {}

for video,frames in dic.items() :
  objets_concat = ''
  objets_concat_unique = ''
  for frame, mots in frames.items() :
    for mot, proba in mots.items() :
      m = mot + ' '
      objets_concat += m
      if (mot not in objets_concat_unique) and (proba > 0.5):
        objets_concat_unique += m

  dic_obj[video] = objets_concat
  dic_obj_u[video] = objets_concat_unique

# inclure dans le df les colonnes contenant les mots concat et concats uniques
def f(x) :
  try :
    obj = dic_obj['video'+x+'.mp4']
  except :
    obj = '0'
  return obj

def f_u(x) :
  try :
    obj = dic_obj_u['video'+x+'.mp4']
  except :
    obj = '0'
  return obj

df['objets_concat'] = df['name'].apply(lambda x : f(x))
df['objets_concat_unique'] = df['name'].apply(lambda x : f_u(x))

df = df[df.new_label != 'not done yet']

<ipython-input-300-6088f3adba40>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['objets_concat'] = df['name'].apply(lambda x : f(x))
<ipython-input-300-6088f3adba40>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['objets_concat_unique'] = df['name'].apply(lambda x : f_u(x))


In [ ]:
# Créer les train et test
df['label'] = df['label'].astype('category')
X_train, X_test, y_train, y_test = train_test_split(df['objets_concat_unique'], df['label'], test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

# Charger le modèle Sentence-Transformers
sentencebert_model = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Encoder les textes avec Sentence-Transformers
train_embeddings = sentencebert_model.encode(X_train.tolist(), convert_to_tensor=True)
test_embeddings = sentencebert_model.encode(X_test.tolist(), convert_to_tensor=True)
valid_embeddings = sentencebert_model.encode(X_val.tolist(), convert_to_tensor=True)

In [ ]:
X_train = train_embeddings.cpu().data.numpy()
X_test = test_embeddings.cpu().data.numpy()
X_val = valid_embeddings.cpu().data.numpy()

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train,y_train)
tfsvmpred = clf.predict(X_test)
print(accuracy_score(y_test,tfsvmpred))


predovr = OneVsRestClassifier(SVC(random_state=0)).fit(X_train, y_train).predict(X_test)
accuracy_score(y_test,predovr)

0.4129353233830846


0.40049751243781095

In [ ]:
def pti_reseau_clas(size_in,size_out):

  x_in = Input(size_in)
  x = x_in

  layers = [256, 256, 64]

  for c in layers :
    x = Dense(c)(x)
    # x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.5)(x)

  x_out = Dense(size_out, activation='softmax')(x)

  model = Model(inputs = x_in, outputs = x_out)
  opt = AdamW()

  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)

  return model

model=pti_reseau_clas(size_in = X_train.shape[1],size_out=df.label.nunique())
model.summary()

Model: "model_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_48 (InputLayer)       [(None, 768)]             0         
                                                                 
 dense_140 (Dense)           (None, 256)               196864    
                                                                 
 re_lu_94 (ReLU)             (None, 256)               0         
                                                                 
 dropout_77 (Dropout)        (None, 256)               0         
                                                                 
 dense_141 (Dense)           (None, 256)               65792     
                                                                 
 re_lu_95 (ReLU)             (None, 256)               0         
                                                                 
 dropout_78 (Dropout)        (None, 256)               0  

In [ ]:
batch_size = 128
epochs=15
hist = model.fit(X_train, pd.get_dummies(y_train),
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(X_test, pd.get_dummies(y_test)))

Epoch 1/15
10/10 [==============================] - 3s 29ms/step - loss: 2.5324 - accuracy: 0.2114 - val_loss: 2.2433 - val_accuracy: 0.2935
Epoch 2/15
10/10 [==============================] - 0s 9ms/step - loss: 2.3924 - accuracy: 0.2629 - val_loss: 2.2265 - val_accuracy: 0.3184
Epoch 3/15
10/10 [==============================] - 0s 9ms/step - loss: 2.2829 - accuracy: 0.2761 - val_loss: 2.1535 - val_accuracy: 0.3159
Epoch 4/15
10/10 [==============================] - 0s 8ms/step - loss: 2.1975 - accuracy: 0.3002 - val_loss: 2.1133 - val_accuracy: 0.3284
Epoch 5/15
10/10 [==============================] - 0s 8ms/step - loss: 2.1677 - accuracy: 0.2910 - val_loss: 2.1107 - val_accuracy: 0.3383
Epoch 6/15
10/10 [==============================] - 0s 8ms/step - loss: 2.1432 - accuracy: 0.3027 - val_loss: 2.0543 - val_accuracy: 0.3483
Epoch 7/15
10/10 [==============================] - 0s 8ms/step - loss: 2.1191 - accuracy: 0.3159 - val_loss: 2.0483 - val_accuracy: 0.3980
Epoch 8/15
10/10 [=

In [ ]:
loss, acc = model.evaluate(X_val, pd.get_dummies(y_val),  verbose=0)
print('The accuracy on the test set is ',(acc*100),'%')

The accuracy on the test set is  38.05970251560211 %


In [ ]:
Y = pd.get_dummies(y_val)
ind = Y[Y.Education == 0].reset_index().index
x_val_woe = X_val[ind]

In [ ]:
loss, acc = model.evaluate(x_val_woe, Y[Y.Education == 0],  verbose=0)
print('The accuracy on the test set is ',(acc*100),'%')

The accuracy on the test set is  5.198776721954346 %


In [ ]:
liste = np.argmax(model.predict(X_train), axis = 1)
dic = {}
for elem in liste:
  if elem in dic:
    dic[elem] +=1
  else:
    dic[elem] = 1

43/43 [==============================] - 0s 2ms/step


In [ ]:
dic

{2: 917, 4: 213, 3: 92, 6: 110, 10: 35}

In [ ]:
np.sum(Y, axis=0)

Autos & Vehicles           4
Comedy                    16
Education                129
Entertainment             63
Film & Animation          71
Gaming                     6
Howto & Style             28
Music                     13
News & Politics           17
Nonprofits & Activism     17
People & Blogs            60
Pets & Animals             8
Science & Technology      11
Sports                     6
Travel & Events            7
dtype: int64

## Embeddings à la mano

In [ ]:
str(dic)[:200]

"{'video-AZS5cgybKcI.mp4': {'94': {'car': 0.44}, '95': {'car': 0.33}, '96': {'car': 0.43}, '97': {'truck': 0.35, 'car': 0.61}, '98': {'traffic light': 0.26, 'person': 0.37, 'car': 0.77}, '99': {'traffi"

In [ ]:
# indexer les mots
dic_obj = {}
index = 0
for video,frames in dic.items() :
  for frame, mots in frames.items() :
    for mot, proba in mots.items() :
      if (mot not in dic_obj):
        dic_obj[mot] = index
        index += 1

In [ ]:
def crop_name(nom):
  return nom[5:-4]
crop_name('video-AZS5cgybKcI.mp4')

In [ ]:
import copy as copy
from tqdm import tqdm

# pour chaque video on initialise la liste d'apparition de chaque mots
dic_obj_full_0 = {}
for mot in dic_obj.keys():
  dic_obj_full_0 [mot] = [0]

# création du dico dans lequel se trouve la proba max de chaque mot pour chaque video
dic_prob_mot = {}
for video, frames in tqdm(dic.items()) :
  copy_dic_obj_full_0 = copy.copy(dic_obj_full_0)
  for frames, mots in frames.items() :
    for mot, proba in mots.items():
      copy_dic_obj_full_0[mot].append(proba)

  dic_max = {}
  for mot in copy_dic_obj_full_0.keys() :
    dic_max[mot] = max(copy_dic_obj_full_0[mot])

  dic_prob_mot[crop_name(video)] = dic_max

100%|██████████| 2279/2279 [04:05<00:00,  9.30it/s]


In [ ]:
import copy as copy
from tqdm import tqdm

# pour chaque video on initialise la liste d'apparition de chaque mots
dic_obj_full_0 = {}
for mot in dic_obj.keys():
  dic_obj_full_0 [mot] = [0]

# création du dico dans lequel se trouve la proba max de chaque mot pour chaque video
dic_avg_prob_mot = {}
for video, frames in tqdm(dic.items()) :
  copy_dic_obj_full_0 = copy.copy(dic_obj_full_0)
  for frames, mots in frames.items() :
    for mot, proba in mots.items():
      copy_dic_obj_full_0[mot].append(proba)

  dic_avg = {}
  for mot in copy_dic_obj_full_0.keys() :
    # valeur moyenne
    dic_avg[mot] = sum(copy_dic_obj_full_0[mot])/len(copy_dic_obj_full_0[mot])


  dic_avg_prob_mot[crop_name(video)] = dic_avg

100%|██████████| 2279/2279 [02:11<00:00, 17.36it/s]


In [ ]:
dic_avg_prob_mot['-KHQk1_Vq69E']

{'car': 0.5021428571428571,
 'truck': 0.31521739130434784,
 'traffic light': 0.43059701492537306,
 'person': 0.5221948717948717,
 'bus': 0.6763636363636363,
 'tv': 0.34643874643874656,
 'cat': 0.4702660753880261,
 'skateboard': 0.365,
 'bottle': 0.46142857142857135,
 'dog': 0.5802080443828023,
 'vase': 0.35951219512195115,
 'cup': 0.35580645161290325,
 'fire hydrant': 0.4539999999999999,
 'snowboard': 0.2,
 'cake': 0.3325,
 'train': 0.3295238095238095,
 'airplane': 0.3014285714285715,
 'bench': 0.34944444444444445,
 'backpack': 0.336923076923077,
 'chair': 0.48311475409836047,
 'dining table': 0.30428571428571427,
 'potted plant': 0.305,
 'teddy bear': 0.38494117647058823,
 'bird': 0.4120588235294118,
 'bicycle': 0.7539473684210529,
 'tennis racket': 0.329,
 'horse': 0.4567961165048543,
 'cow': 0.45551724137931027,
 'couch': 0.24142857142857146,
 'bed': 0.145,
 'bowl': 0.165,
 'mouse': 0.28700000000000003,
 'sheep': 0.4700000000000001,
 'cell phone': 0.19666666666666668,
 'laptop': 0.4

In [ ]:
df = df.drop(df[df['name'] == '-_YoeHOTJBI4'].index)

In [ ]:
import pickle
def save_embeddings(all_embe, folder_path, version):
  # save all embeddings
  with open(folder_path + 'all_embeddings_'+version+'.pkl', 'wb') as ff:
    pickle.dump(all_embe, ff)

def getEmbesDF(path):
  with open(path, "rb") as fp:
      embeddings = pickle.load(fp)
  df_embes = pd.DataFrame.from_dict((embeddings), orient='index')
  return df_embes

In [ ]:
FolderOUT = "/content/gdrive/MyDrive/Projet_Multimedia/download/Video_Embeddings/Video_Embeddings/"
save_embeddings(dic_prob_mot, FolderOUT, "maxProba")
save_embeddings(dic_avg_prob_mot, FolderOUT, "avgProba")

In [ ]:
df_YOLO_avgProba = getEmbesDF(FolderOUT + "all_embeddings_avgProba.pkl")
df2 = df[['name', 'label']]
good = df2.set_index('name').join(df_YOLO_avgProba)

In [ ]:
good

In [ ]:
# créer les embeddings
embeddings = []
def add_embedding(x):
  embeddings.append([0]*len(dic_obj))
  for mot, index in dic_obj.items():
      embeddings[-1][index] = dic_prob_mot['video' + x + '.mp4'][mot]

df['name'].apply(lambda x : add_embedding(x))
embeddings = np.array(embeddings)

## Test de modèles sur les embeddings

In [ ]:
# Créer les train et test
df['label'] = df['label'].astype('category')
X_train, X_test, y_train, y_test = train_test_split(embeddings, df['label'], test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

### SVC

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train,y_train)
tfsvmpred = clf.predict(X_test)
accuracy_score(y_test,tfsvmpred)


predovr = OneVsRestClassifier(SVC(random_state=0)).fit(X_train, y_train).predict(X_test)
accuracy_score(y_test,predovr)

0.11403508771929824

### Réseau pas très profond

In [ ]:
def pti_reseau_clas(size_in,size_out):

  x_in = Input(size_in)
  x = x_in

  layers = [256,256,64]

  for c in layers :
    x = Dense(c)(x)
    # x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dropout(0.5)(x)

  x_out = Dense(size_out, activation='softmax')(x)

  model = Model(inputs = x_in, outputs = x_out)
  opt = AdamW()

  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)

  return model

model=pti_reseau_clas(size_in = len(dic_obj),size_out=df.label.nunique())
model.summary()

Model: "model_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_45 (InputLayer)       [(None, 80)]              0         
                                                                 
 dense_128 (Dense)           (None, 256)               20736     
                                                                 
 re_lu_85 (ReLU)             (None, 256)               0         
                                                                 
 dropout_68 (Dropout)        (None, 256)               0         
                                                                 
 dense_129 (Dense)           (None, 256)               65792     
                                                                 
 re_lu_86 (ReLU)             (None, 256)               0         
                                                                 
 dropout_69 (Dropout)        (None, 256)               0  

In [ ]:
batch_size = 256
epochs=20
hist = model.fit(X_train, pd.get_dummies(y_train),
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(X_test, pd.get_dummies(y_test)))

Epoch 1/20
6/6 [==============================] - 4s 137ms/step - loss: 2.6265 - accuracy: 0.1880 - val_loss: 2.3624 - val_accuracy: 0.2829
Epoch 2/20
6/6 [==============================] - 0s 21ms/step - loss: 2.4180 - accuracy: 0.3058 - val_loss: 2.2961 - val_accuracy: 0.2829
Epoch 3/20
6/6 [==============================] - 0s 20ms/step - loss: 2.3510 - accuracy: 0.2970 - val_loss: 2.2977 - val_accuracy: 0.2829
Epoch 4/20
6/6 [==============================] - 0s 21ms/step - loss: 2.3357 - accuracy: 0.2992 - val_loss: 2.2683 - val_accuracy: 0.2829
Epoch 5/20
6/6 [==============================] - 0s 22ms/step - loss: 2.3040 - accuracy: 0.2985 - val_loss: 2.2282 - val_accuracy: 0.2829
Epoch 6/20
6/6 [==============================] - 0s 30ms/step - loss: 2.2618 - accuracy: 0.3197 - val_loss: 2.2451 - val_accuracy: 0.2829
Epoch 7/20
6/6 [==============================] - 0s 21ms/step - loss: 2.2545 - accuracy: 0.3219 - val_loss: 2.2351 - val_accuracy: 0.2829
Epoch 8/20
6/6 [==========

In [ ]:
loss, acc = model.evaluate(X_val, pd.get_dummies(y_val),  verbose=0)
print('The accuracy on the test set is ',(acc*100),'%')

The accuracy on the test set is  28.289473056793213 %
